In [1]:
import os
import pandas as pd
from PyPDF2 import PdfReader
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from pprint import pprint

/home/keval/envs/topicmodeling/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


# Reading pdf files from inside pdfs folder and storing inside a dataframe

In [ ]:
files = [f for f in os.listdir("pdfs/") if os.path.isfile("pdfs/"+f) and f.endswith('.pdf')]

data = []

for name in files:
    reader = PdfReader("pdfs/"+name)
    number_of_pages = len(reader.pages)
    text = ""
    for i in range(number_of_pages):
        page = reader.pages[i]
        tt = page.extract_text()
        text +=tt
    data.append(text)
    
df = pd.DataFrame(data, index=None)
df

# Converting the text from pdfs into words

In [ ]:
def doc_to_words(doc):
    return(gensim.utils.simple_preprocess(str(doc), deacc=True))

for idx, item in enumerate(df[0]):
    df[0][idx]=list(doc_to_words(item))
df

# Printing number of words inside each pdfs

In [ ]:
for doc in df[0]:
    print(len(doc))

# Removing stopwords from the words list

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'group', 'financial', 'report'])

def remove_stopwords(texts):
    return [word for word in texts if word not in stop_words]

for idx, item in enumerate(df[0]):
    df[0][idx]=remove_stopwords(item)
    
df

# Printing length of words after removing stopwords

In [ ]:
for doc in df[0]:
    print(len(doc))

# Creating unique token for each word

In [ ]:
for doc in df[0]:
    id2word = corpora.Dictionary([doc])
    id2word.token2id
    corpus = [id2word.doc2bow(text) for text in [doc]]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    top_topics = pd.DataFrame(lda_model.print_topics())
    top_topics = top_topics.set_index(0)
    print(top_topics)
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))

    coherence_model_lda = CoherenceModel(model=lda_model, texts=[doc], dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)


In [ ]:
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

# pyLDAvis.display(vis)